## Setup

Begin by installing and importing some necessary libraries

In [3]:
# The way this tutorial uses the `TimeDistributed` layer requires TF>=2.10
!pip install -U "tensorflow>=2.10.0"

In [4]:
!pip install remotezip tqdm opencv-python
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from sklearn.preprocessing import LabelEncoder

import cv2 # process video files
from google.colab.patches import cv2_imshow

import imageio
from tensorflow_docs.vis import embed #for embedding data in a Jupyter notebook

## Download dataset



In [6]:
drive.mount('/content/drive')
dataset_file_path = "/content/drive/My Drive/"

dataset_filepath = dataset_file_path+"final_exam/input_data.csv"
df = pd.read_csv(dataset_filepath)

Mounted at /content/drive


## Create frames from each video file



Using pandas library to process dataframe and returen to the seconds.

In [7]:
def convert_time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

df["start_time_seconds"]=df["Start Time"].apply(convert_time_to_seconds)
df["end_time_seconds"]=df["End Time"].apply(convert_time_to_seconds)

In [8]:
#Extract frame from video
def extract_frame(path,start_time,end_time):
  frame_list = []
  SEQUENCE_LENGTH = 30

  # Open the video file for reading
  video_reader = cv2.VideoCapture(path)

  # Get the frames per second (fps), start_frame, and end_frame for the specified video duration
  fps = video_reader.get(cv2.CAP_PROP_FPS)
  start_frame = int(start_time*fps)
  end_frame = int(end_time*fps)
  interval_frame = end_frame - start_frame

  #calculate the skip frame number formula total duration of interval frame / sequence length
  skip_frame_window = max(int(interval_frame/SEQUENCE_LENGTH),1)

  # Get the height and width of the video frames
  height = video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT)
  width = video_reader.get(cv2.CAP_PROP_FRAME_WIDTH)
  print(f'fps:{fps}, interval_frame:{interval_frame}, height:{height}, width:{width}')


  # Set the video reader to the start frame
  video_reader.set(cv2.CAP_PROP_POS_FRAMES,start_frame)
  current_frame = start_frame

  # Iterate through frames in the specified interval
  while video_reader.isOpened() and current_frame <= end_frame:
    ret, frame = video_reader.read()

    if not ret:
      break

    # Convert the frame from BGR to RGB
    frame_rgb_opencv = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Resize the frame to a specific size (720x1280)
    resized_frame = cv2.resize(frame_rgb_opencv,(720,1280))
    # Append the resized frame to the frame_list
    frame_list.append(resized_frame)
    # Set the video reader to the next frame position
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, current_frame + skip_frame_window )

    # Increment the current_frame by skip_frame_window
    current_frame += skip_frame_window

    #current_frame += 1

  # Release the video reader and close any remaining OpenCV windows
  video_reader.release()
  cv2.destroyAllWindows()

  # Return the list of extracted frames
  return frame_list;

In [ ]:
X = [] #Save the frame
Y = [] #Save the lable

for index,row in df.iterrows():
  start_time = row["start_time_seconds"]
  end_time = row["end_time_seconds"]
  path = dataset_file_path + row["GD_Path"] # Get the vidoe file path
  label = row["Movement Name"] # Get the Movement lable


  Y.append(label)
  X.append(np.asarray(extract_frame(path,start_time,end_time)))

fps:24.12082514734774, interval_frame:144, height:480.0, width:640.0
fps:30.0, interval_frame:150, height:360.0, width:204.0
fps:30.0, interval_frame:120, height:360.0, width:204.0
fps:30.0, interval_frame:150, height:360.0, width:204.0
fps:15.033, interval_frame:75, height:480.0, width:640.0
fps:30.0, interval_frame:600, height:720.0, width:1280.0
fps:30.0, interval_frame:240, height:720.0, width:1280.0
fps:15.0, interval_frame:180, height:720.0, width:1280.0
fps:23.954, interval_frame:72, height:240.0, width:320.0
fps:23.954, interval_frame:96, height:240.0, width:320.0
fps:23.954, interval_frame:168, height:240.0, width:320.0
fps:23.954, interval_frame:432, height:240.0, width:320.0
fps:23.954, interval_frame:287, height:240.0, width:320.0
fps:23.954, interval_frame:119, height:240.0, width:320.0
fps:23.954, interval_frame:120, height:240.0, width:320.0
fps:30.0, interval_frame:300, height:480.0, width:640.0
fps:30.0, interval_frame:420, height:480.0, width:640.0
fps:30.0, interval_

In [ ]:
def to_gif(images):
  converted_images = np.clip(images, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=10)
  return embed.embed_file('./animation.gif')

In [ ]:
to_gif(X[0])